# <font color=blue>Assignments for "Overfitting and Regularization"</font>

In this assignment, you'll continue working with the house prices data. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

In [1]:
# These are the libraries we use
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse
from sklearn.model_selection import GridSearchCV




import warnings
warnings.filterwarnings('ignore')
house_prices_df = pd.read_csv("https://djl-lms-assets.s3.eu-central-1.amazonaws.com/datasets/house_prices.csv", sep = ";")
house_prices_df.head()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


- Reimplement your model from the previous lesson.

In [2]:
house = house_prices_df.drop(['Id','PoolQC','MiscFeature', 'Alley','Fence', 'FireplaceQu', 'LotFrontage'], axis=1)
house = house.drop(house[(house['GrLivArea']>4000) & (house['SalePrice']<300000)].index)
house = house.drop(house[(house["TotalBsmtSF"]>6000) & (house["SalePrice"]<200000)].index)
house = house.drop(house[(house["TotRmsAbvGrd"]>12) & (house["SalePrice"]<250000)].index)

In [3]:
house['PoolArea'] = pd.get_dummies(house['PoolArea'],drop_first=True)
house['GarageArea'] = pd.get_dummies(house['GarageArea'],drop_first=True)
house['TotalBsmtSF'] = pd.get_dummies(house['TotalBsmtSF'],drop_first=True)
house['2ndFlrSF'] = pd.get_dummies(house['2ndFlrSF'],drop_first=True)
house['Fireplaces'] = pd.get_dummies(house['Fireplaces'],drop_first=True)

In [4]:
house['GarageYrBlt'] = house['GarageYrBlt'].fillna(house['YearBuilt'])
house['MasVnrArea'] = house['MasVnrArea'].fillna(house['MasVnrArea'].mean())

In [5]:
house['SalePrice'] = np.log(house['SalePrice'])

In [6]:
cols_fillna = [ 'MasVnrType','GarageQual','GarageCond','GarageFinish','GarageType', 'Electrical','KitchenQual','SaleType', 
               'Functional', 'Exterior2nd', 'Exterior1st','BsmtExposure','BsmtCond','BsmtQual','BsmtFinType1','BsmtFinType2','MSZoning', 'Utilities']

# replace 'NaN' with 'None' in these columns
for col in cols_fillna:
    house[col].fillna('None',inplace=True)

In [7]:
from sklearn.preprocessing import LabelEncoder
categorical_feature_mask = house.dtypes==object
categorical_cols = house.columns[categorical_feature_mask].tolist()
labelencoder = LabelEncoder()
house[categorical_cols] = house[categorical_cols].apply(lambda col: labelencoder.fit_transform(col.astype(str)))

In [8]:
house['has_pool'] = house['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
house['has_garage'] = house['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
house['has_bsmt'] = house['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
house['has_2ndfloor'] = house['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
house['has_fireplace'] = house['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

In [9]:
Y = house['SalePrice']
X = house.drop('SalePrice', axis=1)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 123)

In [11]:
lrm = LinearRegression()
lrm.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [12]:
y_preds_train = lrm.predict(X_train)
y_preds_test = lrm.predict(X_test)

In [13]:
print("R-squared of the model in training set is: {}".format(lrm.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(lrm.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in training set is: 0.9209879026950388
-----Test set statistics-----
R-squared of the model in test set is: 0.8950032452257967
Mean absolute error of the prediction is: 0.09062337274971963
Mean squared error of the prediction is: 0.016551713433027696
Root mean squared error of the prediction is: 0.12865346257690732
Mean absolute percentage error of the prediction is: 0.7613122052970146


- Try OLS, Lasso, Ridge and ElasticNet regressions using the same model specification. This time, you need to do **k-fold cross-validation** to choose the best hyperparameter values for your models. Which model is the best? Why?

### Ridge

In [14]:
from sklearn.linear_model import Ridge
ridgeregr = Ridge(alpha=1000) 
ridgeregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = ridgeregr.predict(X_train)
y_preds_test = ridgeregr.predict(X_test)

print("R-squared of the model in training set is: {}".format(ridgeregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(ridgeregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in training set is: 0.9031436587578683
-----Test set statistics-----
R-squared of the model in test set is: 0.8775313633491184
Mean absolute error of the prediction is: 0.0968233814999105
Mean squared error of the prediction is: 0.019305985053901995
Root mean squared error of the prediction is: 0.13894597890511978
Mean absolute percentage error of the prediction is: 0.8133482312007415


In [15]:
params = {'alpha': [10**i for i in range(1, 10)] + [round(0.1**i, 10) for i in range(10)]}


In [16]:
ridge_grid = GridSearchCV(Ridge(), param_grid=params, cv=10)

ridge_grid.fit(X_train, y_train)
print(ridge_grid.best_params_)

pd.DataFrame(ridge_grid.cv_results_)[['param_alpha', 'mean_test_score']].sort_values('mean_test_score', ascending=False).head()

{'alpha': 100}


,param_alpha,mean_test_score
1,100,0.904659
0,10,0.903939
9,1,0.903271
10,0.1,0.902935
11,0.01,0.902876


### Lasso

In [17]:
from sklearn.linear_model import Lasso

lassoregr = Lasso(alpha=1000) 
lassoregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = lassoregr.predict(X_train)
y_preds_test = lassoregr.predict(X_test)

print("R-squared of the model in training set is: {}".format(lassoregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(lassoregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in training set is: 0.0
-----Test set statistics-----
R-squared of the model in test set is: -0.0005708810911657647
Mean absolute error of the prediction is: 0.30244956956080055
Mean squared error of the prediction is: 0.15773023203305608
Root mean squared error of the prediction is: 0.39715265583029413
Mean absolute percentage error of the prediction is: 2.521020368882766


In [18]:

lasso_grid = GridSearchCV(Lasso(), param_grid=params, cv=10)

lasso_grid.fit(X_train, y_train)
print(lasso_grid.best_params_)
pd.DataFrame(lasso_grid.cv_results_)[['param_alpha', 'mean_test_score']].sort_values('mean_test_score', ascending=False).head()

{'alpha': 0.001}


,param_alpha,mean_test_score
12,0.001,0.903994
13,0.0001,0.903147
14,1e-05,0.902909
15,1e-06,0.902873
16,1e-07,0.902869


### ElasticNet

In [19]:
from sklearn.linear_model import ElasticNet

elasticregr = ElasticNet() 
elasticregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = elasticregr.predict(X_train)
y_preds_test = elasticregr.predict(X_test)

print("R-squared of the model in training set is: {}".format(elasticregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(elasticregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in training set is: 0.8346839449230855
-----Test set statistics-----
R-squared of the model in test set is: 0.8124707610160296
Mean absolute error of the prediction is: 0.1227675931027577
Mean squared error of the prediction is: 0.029562153903246593
Root mean squared error of the prediction is: 0.17193648217654853
Mean absolute percentage error of the prediction is: 1.028980314440785


In [20]:
elastic_grid = GridSearchCV(ElasticNet(), param_grid=params, cv=10)

elastic_grid.fit(X_train, y_train)
print(elastic_grid.best_params_)
pd.DataFrame(elastic_grid.cv_results_)[['param_alpha', 'mean_test_score']].sort_values('mean_test_score', ascending=False).head()

{'alpha': 0.001}


,param_alpha,mean_test_score
12,0.001,0.903939
13,0.0001,0.903057
14,1e-05,0.902892
15,1e-06,0.902871
16,1e-07,0.902869


According to the results, ElasticNet's train and test set performances are somewhere between Lasso's and Ridge's.

